In [ ]:
window segmentation of edf file

In [ ]:
def read_edf(files_train):
    time_window = 8
    time_step = 4

    if os.path.exists('/kaggle/input/mit-chb-processed/signal_samples.npy')&os.path.exists('/kaggle/input/mit-chb-processed/is_sz.npy'):
        array_signals=np.load('/kaggle/input/mit-chb-processed/signal_samples.npy')
        array_is_sz=np.load('/kaggle/input/mit-chb-processed/is_sz.npy')
    else:
        p = 0.01
        counter = 0
        step_window = 0
        for temp_f in files_train:
            temp_edf =  mne.io.read_raw_edf(temp_f)
            temp_labels = temp_edf.ch_names
            if sum([any([0 if re.match(c, l) is None else 1 for l in temp_edf.ch_names]) for c in ch_labels])==len(ch_labels):
                time_window = 8
                time_step = 4
                fs = int(1/(temp_edf.times[1]-temp_edf.times[0]))
                step_window = time_window*fs
                step = time_step*fs

                temp_is_sz = np.zeros((temp_edf.n_times,))
                if os.path.exists(temp_f+'.seizures'):
                    temp_annotation = wfdb.rdann(temp_f, 'seizures')
                    for i in range(int(temp_annotation.sample.size/2)):
                        temp_is_sz[temp_annotation.sample[i*2]:temp_annotation.sample[i*2+1]]=1
                temp_len = temp_edf.n_times

                temp_is_sz_ind = np.array(
                    [temp_is_sz[i*step:i*step+step_window].sum()/step_window for i in range((temp_len-step_window)//step)]
                )

                temp_0_sample_size = round(p*np.where(temp_is_sz_ind==0)[0].size)
                temp_1_sample_size = np.where(temp_is_sz_ind>0)[0].size

                counter = counter + temp_0_sample_size + temp_1_sample_size
            temp_edf.close()
            return counter, ch_labels, step_window

def extract_edf2(files_train, counter, ch_labels, step_window):
        array_signals = np.zeros((counter, len(ch_labels), step_window), dtype=np.float32)
        array_is_sz = np.zeros(counter, dtype=bool)

        counter = 0
        for n, temp_f in enumerate(tqdm.tqdm(files_train)):
            to_log = 'No. {}: Reading. '.format(n)
            temp_edf =  mne.io.read_raw_edf(temp_f)
            temp_labels = temp_edf.ch_names
            n_label_match = sum([any([0 if re.match(c, l)==None else 1 for l in temp_edf.ch_names]) for c in ch_labels])
            if n_label_match==len(ch_labels):
                ch_mapping = {sorted([l for l in temp_edf.ch_names if re.match(c, l)!=None ])[0]:c for c in ch_labels}
                temp_edf.rename_channels(ch_mapping)
                #temp_edf = temp_edf.pick(ch_labels)

                temp_is_sz = np.zeros((temp_edf.n_times,))
                temp_signals = temp_edf.get_data(picks=ch_labels)*1e6

                if os.path.exists(temp_f+'.seizures'):
                    to_log = to_log+'sz exists.'
                    temp_annotation = wfdb.rdann(temp_f, 'seizures')
                    for i in range(int(temp_annotation.sample.size/2)):
                        temp_is_sz[temp_annotation.sample[i*2]:temp_annotation.sample[i*2+1]]=1
                else:
                    to_log = to_log+'No sz.'

                temp_len = temp_edf.n_times

                time_window = 8
                time_step = 4
                fs = int(1/(temp_edf.times[1]-temp_edf.times[0]))
                step_window = time_window*fs
                step = time_step*fs

                temp_is_sz_ind = np.array(
                    [temp_is_sz[i*step:i*step+step_window].sum()/step_window for i in range((temp_len-step_window)//step)]
                )
                del temp_is_sz

                temp_0_sample_size = round(p*np.where(temp_is_sz_ind==0)[0].size)
                temp_1_sample_size = np.where(temp_is_sz_ind>0)[0].size

                # sz data
                temp_ind = list(np.where(temp_is_sz_ind>0)[0])
                for i in temp_ind:
                    array_signals[counter, :, :] = temp_signals[:, i*step:i*step+step_window]
                    array_is_sz[counter] = True
                    counter = counter+1

                # no sz data
                temp_ind = random.sample(list(np.where(temp_is_sz_ind==0)[0]), temp_0_sample_size)
                for i in temp_ind:
                    array_signals[counter, :, :] = temp_signals[:, i*step:i*step+step_window]
                    array_is_sz[counter] = False
                    counter = counter+1

                to_log += '{} signals added: {} w/o sz, {} w/ sz.'.format(
                    temp_0_sample_size+temp_1_sample_size, temp_0_sample_size, temp_1_sample_size
                )
                print("array_signals",array_signals.shape, array_signals[:2], "temp_signals",temp_signals.shape, temp_signals[:2])

            else:
                to_log += 'Not appropriate channel labels. Reading skipped.'.format(n)


            temp_edf.close()

        np.save('signal_samples', array_signals)
        np.save('is_sz', array_is_sz)


In [ ]:
# same as above but with comments
def extract_edf2(folder, n_samples, n_channel_labels, window_size):
    if os.path.exists("10sec_signals.npy") and os.path.exists("10sec_labels.npy"):
        print("File 10sec_signals.npy and seizure_labels.npy exists.")
        return
    signals_np = np.zeros((n_samples, n_channel_labels, window_size), dtype=np.float32)
    labels_np = np.zeros(n_samples, dtype=bool)


    # Read  edf, rename file channel to match names from channel labels list. When files have multiple channel names, only first is picked.
    for number, file in enumerate(tqdm.tqdm(folder)):
        log = f"Reading file {number} "
        edf_file = mne.io.read_raw_edf(file, preload=False)

        n_label_match = sum([any([0 if re.match(ch, ch_name) is None else 1 for ch_name in edf_file.ch_names]) for ch in channel_labels])
        if n_label_match == len(channel_labels):
            dict_ch_name = {sorted([ch_name for ch_name in edf_file.ch_names if re.match(ch, ch_name) is not None])[0]: ch for ch in channel_labels}
            edf_file.rename_channels(dict_ch_name)

            # Retrieve EEG (in microvolts) ,  annotations
            has_seizure = np.zeros((edf_file.n_times,))
            signals_ = edf_file.get_data(picks=channel_labels) * TO_MICROVOLTS

            if os.path.exists(file + '.seizures'):
                log += "positive seizure"
                has_annotation = wfdb.rdann(file, 'seizures')
                for idx in range(int(has_annotation.sample.size / 2)):
                    has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1
            else:
                log += "negative seizure"

            # create seizure index, and calculate proportion of signal which shows seizures
            sampling_freq = int(1/(edf_file.times[1] - edf_file.times[0]))
            print("sampling frequency ",sampling_freq)
            window_stride = int(sampling_freq * STEP_TIME)
            print(f"Window stride: {window_stride}, Window size: {window_size}")
            is_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_file.n_times - window_size) // window_stride)])

            # populate numpy array with EEG , annotation data
            noseizure_n_size = round(SEIZURE_PROPORTION * np.where(is_seizure_idx == 0)[0].size)
            seizure_n_size = np.where(is_seizure_idx > 0)[0].size


            # non seizure data are by far larger than seizure data. To avoid overfitting, and bias, non seizure data is randomly subsampled. This prevents model from being overwhelmed by large non seizure data
            count = 0
            # no seizure
            temp_negative = random.sample(list(np.where(is_seizure_idx == 0)[0]), noseizure_n_size)
            for value in temp_negative:
                start_idx = value * window_stride
                end_idx = start_idx + window_size
                segment = signals_[:, start_idx:end_idx]
                print(f"Segment shape: {segment.shape}")  # Debug print
                if segment.shape == (n_channel_labels, window_size):
                    signals_np[count, :, :] = segment
                    labels_np[count] = False
                    count += 1
                else:
                    print(f"Skipping segment due to shape mismatch: {segment.shape}")

            # seizure
            temp_positive = list(np.where(is_seizure_idx > 0)[0])
            for value in temp_positive:
                start_idx = value * window_stride
                end_idx = start_idx + window_size
                segment = signals_[:, start_idx:end_idx]
                print(f"Segment shape: {segment.shape}")  # Debug print
                if segment.shape == (n_channel_labels, window_size):
                    signals_np[count, :, :] = segment
                    labels_np[count] = True
                    count += 1
                else:
                    print(f"Skipping segment due to shape mismatch: {segment.shape}")

            print(f"{noseizure_n_size + seizure_n_size} EEG signals added {seizure_n_size} with, {noseizure_n_size} without seizures")
        else:
            print(f"Unable to read {file}")

        edf_file.close()

    np.save('eeg_signals', signals_np)
    np.save('seizure_labels', labels_np)

In [ ]:
count1, ch_labels1, step_window1 = read_edf(train_files)
extract_edf2(train_files, count1, ch_labels1, step_window1)